# Numba demos snippets

Numba is a compiler for Python array and numerical functions.

## How does Numba work?
Numba reads the Python bytecode for a decorated function and combines this with information about the types of the input arguments to the function. It analyzes and optimizes your code, and finally uses the LLVM compiler library to generate a machine code version of your function, tailored to your CPU capabilities. This compiled version is then used every time your function is called.


In [10]:
import time

import numba
import numpy as np
import pandas as pd
from numba import config, jit, njit, threading_layer

In [11]:
x = np.arange(100).reshape(10, 10)


@jit(nopython=True)
def go_fast(a):  # Function is compiled and runs in machine code
    trace = 0.0
    for i in range(a.shape[0]):
        trace += np.tanh(a[i, i])
    return a + trace


# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
go_fast(x)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
go_fast(x)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.07579612731933594
Elapsed (after compilation) = 2.384185791015625e-05


In [12]:
# Even better time recording:
# measure multiple iterations of execution and, as a result,
# can be made to accommodate for the compilation time in the first execution.
%timeit go_fast(x)

504 ns ± 13.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [13]:
# simple example with pandas DataFrames


def f(x):
    return x * (x - 1)


def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx


@jit
def f_plain(x):
    return x * (x - 1)


@jit
def integrate_f_numba(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f_plain(a + i * dx)
    return s * dx


@jit(nopython=True)
def apply_integrate_f_numba(col_a, col_b, col_N):
    n = len(col_N)
    result = np.empty(n, dtype=np.float64)
    assert len(col_a) == len(col_b) == n
    for i in range(n):
        result[i] = integrate_f_numba(col_a[i], col_b[i], col_N[i])
    return result


def compute_numba(df):
    result = apply_integrate_f_numba(df["a"].to_numpy(), df["b"].to_numpy(), df["N"].to_numpy())
    return pd.Series(result, index=df.index, name="result")

In [14]:
rand_df = pd.DataFrame(
    {"a": np.random.randn(1000), "b": np.random.randn(1000), "N": np.random.randint(100, 1000, (1000)), "x": "x"}
)

In [15]:
%timeit rand_df.apply(lambda x: integrate_f(x['a'], x['b'], x['N']), axis=1)

39.5 ms ± 1.72 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%timeit compute_numba(rand_df)

491 µs ± 6.12 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Parallel logistic regression


In [17]:
@numba.jit(nopython=True, parallel=True)
def logistic_regression(Y, X, w, iterations):
    for i in range(iterations):
        w -= np.dot(((1.0 / (1.0 + np.exp(-Y * np.dot(X, w))) - 1.0) * Y), X)
    return w

## Parallel execution

In [18]:
# set the threading layer before any parallel target compilation
config.THREADING_LAYER = "threadsafe"


@njit(parallel=True)
def foo(a, b):
    return a + b


x = np.arange(10.0)
y = x.copy()

# this will force the compilation of the function, select a threading layer
# and then execute in parallel
foo(x, y)

# demonstrate the threading layer chosen
print("Threading layer chosen: %s" % threading_layer())

LoweringError: Failed in nopython mode pipeline (step: native parfor lowering)
No threading layer could be loaded.
HINT:
One of:
Intel TBB is required, try:
$ conda/pip install tbb
OR
Intel OpenMP is required, try:
$ conda/pip install intel-openmp

File "../../../../../../var/folders/s8/rl4rq8tn2qdc72zdzr0y8pdc0000gn/T/ipykernel_33212/4107096637.py", line 7:
<source missing, REPL/exec in use?>

During: lowering "id=0[LoopNest(index_variable = parfor_index.8, range = (0, a_size0.1, 1))]{1: <ir.Block at /var/folders/s8/rl4rq8tn2qdc72zdzr0y8pdc0000gn/T/ipykernel_33212/4107096637.py (7)>}Var(parfor_index.8, 4107096637.py:7)" at /var/folders/s8/rl4rq8tn2qdc72zdzr0y8pdc0000gn/T/ipykernel_33212/4107096637.py (7)